## Instalação do banco de dados vetorial Milvus 

#### Primeiro passo 
É necessário fazer o download do [docker_compose.yml](https://github.com/milvus-io/milvus/releases/download/v2.2.11/milvus-standalone-docker-compose.yml) 


Ou cole o comando abaixo diretamente no terminal

 - wget https://github.com/milvus-io/milvus/releases/download/v2.2.11/milvus-standalone-docker-compose.yml -O docker-compose.yml
go to docker-compose.yml

* Este arquivo contém a configuração de 3 conteiners necessários para execução do milvus, sendo eles: 
   - etcd: Responsável por setar os volumes para persistência de dados.
   - minio: Responsável por armazenar metadados e outros dados necessários para o funcionamento do milvus
   - standalone: É o container principal que executa o milvus

Execute o docker compose com o comando `docker compose up`

Após ser executado, vc irá notar que seu terminal ficará preso por conta dos containeres estarem em execução

Abra uma outra janela do terminal e derrube a execução com o comando `docker compose down`

Abaixo da configuração do standalone cole o código abaixo no seu arquivo docker compose

Agora iremos modificar o nosso arquivo docker compose para instalar um container do Attu (Um software open-source que nos permite ter acesso a uma interface visual do nosso banco vetorial) 

Abra o aquivo docker compose e você irá visualizar em 'services' a configuração dos 3 containers: etcd, minio, standalone

Abaixo das configurações do standalone insira esta configuração.  
Obs: Nos arquivos yaml a identação importa, então atenção. 

```
attu:
    container_name: attu
    image: zilliz/attu:v2.2.6
    environment:
      MILVUS_URL: milvus-standalone:19530
    ports:
      - "8000:3000"
    depends_on:
      - "standalone"
```      

Execute o comando docker compose up novamente 

### Attu

Agora vamos executar o [Attu](http://localhost:8000) que será a interface gráfica do nosso banco vetorial.

Ele está rodando na porta 8000 do nosso computador  
login: root  
senha: milvus

### Pymilvus

Agora que já temos o nosso container rodando, vamos instalar o pymilvus para persistir alguns dados para teste

In [ ]:
!pip install pymilvus

In [4]:
import os
from pymilvus import(
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection
)

Conecção com o banco

In [5]:
connections.connect('default', host = 'localhost', port = '19530')

Aqui abaixo estamos configurando como será o esquema da nossa coleção de dados.
 - fields: Setamos os campos da nossa coleção onde teremos 3 colunas sendo elas pk, words e embbedings.
 - schema: Passamos como argumento os campos para o nosso schema e podemos especificar uma descrição para essa coleção
 - my_collection: Instanciamos a nossa coleção passando como argumento o nosso schema e setamos o nome dessa coleção

In [6]:
fields = [
    FieldSchema(name='pk', dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name='words', dtype=DataType.VARCHAR, max_length = 50),
    FieldSchema(name='embbendings', dtype=DataType.FLOAT_VECTOR, dim = 300),
]
schema = CollectionSchema(fields, 'Vector Colection')
my_collection = Collection('my_collection', schema)

Aqui iremos setar o index type da coluna embbedings.  
O index type é importante para garantir um bom desempenho e precisão nas consultas de similaridade.  
Parametros:
- 'index_type:IVF_FLAT' - Determina como os vetores serão organizados e indexados. O 'IVF_FLAT' é um tipo de indice que combina a indexação com um estrutura de dados em arvore.  
- metric_type:L2 - Especifica a medida de similaridade usada para comparação entre os vetores. No caso, distancia Euclidiana.  
- nlist:128 - Controla o numero de clusters ou partições usadas pelo indice. O valor padrão é 16384.  

Obs: Quanto maior o valor de nlist, mais clusters serão criados e menor será o número de vetores em cada cluster. Isso pode afetar a precisão das consultas}

In [7]:
index = {
    'index_type':'IVF_FLAT',
    'metric_type':'L2',
    'params': {'nlist':128}
}

my_collection.create_index('embbendings', index)

Status(code=0, message=)

Agora neste passo iremos usar o modelo pré treinado para transformar nossos dados em vetores para armazená-los no milvus.  
Caso tenha interesse em baixar o modelo clique [Aqui](http://www.nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc)  
Obs: Este processo pode ser um pouco demorado dependendo de quantas dimensões o modelo pré treinado possui

In [8]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(os.path.join(os.getcwd(), 'cbow_s300.txt'))

Aqui colocamos algumas palavras aleatórias dentro de uma lista e iteramos sobre elas para primeiro verificar se ela existe no nosso modelo e caso a condição seja satisfeita, transformamos-a em vetores numericos e inserimos cada vetor dentro de uma nova lista para ser usadas futuramente.

In [9]:
words = ['cachorro', 'gato','rei', 'rainha', 'leão', 'selva', 'tigre', 'estudo']
embbending_words = []
for word in words:
    if word in model:
        embbending_words.append(model[word])

Usamos um recurso do python para fazer a atribuição dos vetores obitidos de cada palavra

In [10]:
cachorro, gato, rei, rainha, leão, selva, tigre, estudo = embbending_words

In [11]:
data = [
    [1, 2, 3, 4, 5, 6, 7, 8],
    ['cachorro', 'gato','rei', 'rainha', 'leão', 'selva', 'tigre', 'estudo'],
    [cachorro, gato, rei,  rainha, leão, selva, tigre, estudo]
]

Como usamos um modelo de 300 dimensões verificamos o shape de uma variável para testar

In [34]:
cachorro.shape

(300,)

E por fim inserimos a nossa coleção 8 registros 

In [12]:
my_collection.insert(data)

(insert count: 8, delete count: 0, upsert count: 0, timestamp: 448719281261379586, success count: 8, err count: 0)

Após a inserção usamos o método flush para garantir que os dados sejam persistidos no armazenamento permanente. É útil para que os dados não sejam perdidos, caso haja falha do sistema ou reinicialização do milvus.

In [13]:
my_collection.flush()

O método load permite que os dados sejam carregados na memória para estar disponível para consulta

In [14]:
my_collection.load()